<a href="https://colab.research.google.com/github/Crossme0809/frenzyTechAI/blob/main/fine-tuned-llm-trainer/How_to_Fine_Tune_and_Train_LLMs_With_FAST_GPT_LLM_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安装必要的库

In [1]:
import os
import sys
sys.path.append('..')
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"
from model.llama import Llama

2024-05-14 05:12:49.785847: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 05:12:49.788017: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-14 05:12:49.837149: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 05:12:50.535089: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
import transformers
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7" #指定cuda可见显卡编号
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,

)
from peft import LoraConfig, PeftModel
from transformers import EarlyStoppingCallback

transformers.__version__

'4.40.1'

In [3]:
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

#运行推理

In [5]:
!nvidia-smi #查看显卡占用情况

Tue May 14 05:13:09 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:4F:00.0 Off |                  Off |
| 30%   46C    P2   101W / 300W |  25628MiB / 49140MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:52:00.0 Off |                  Off |
| 30%   

In [11]:
#加载模型

from transformers import AutoModelForCausalLM, AutoTokenizer

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model_path = "/home/jovyan/notebook/LLAMA-3-8B"  # 更改为保存模型的路径,可用范围：MineLLaMa、MineLLaMa-v3、MineLLaMa-v4

model = AutoModelForCausalLM.from_pretrained(model_path,
                                             quantization_config=bnb_config,
                                             device_map={"":4})
                                            # device_map='auto')  #device_map={"": 5}或device_map={"": 6}或device_map={"": 7}，使用前可调用上代码框中的!nvidia-smi查看显卡占用情况
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [12]:
#推理

logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(task="text-generation", 
                model=model, 
                tokenizer=tokenizer,
                max_length=1024,
                # repetition_penalty=1.18,
                # no_repeat_ngram_size=5,
                return_full_text=False,
                temperature=0.8)


In [13]:
user_prompt='Task list from last round: None; Critique: None; Monster: 1 zombie.'
system_prompt='''
In Minecraft, combating with monsters requires weapons and armor.\nThe weapon options are limited to "sword", while the armor includes "helmet", "chestplate", "leggings", and "boots".\nThe materials for swords range from low to high level: wooden swords, stone swords, iron swords, and diamond swords;\nThe materials for armor range from low to high level: iron, diamond.\nThe higher the material level, the greater the attack damage of the weapon and the better the protection effect of the armor.\nHowever, the higher the material level, the more time it costs to collect.\n\nTips: Wooden, stone, iron and diamond are the only levels of sword; iron and diamond are the only levels of armors;\nhelmet, chestplate, leggings and boots are the only types of armors; do not generate information that doesn\'t relate to them.\n\nYou goal is to generate the plan that can defeat all monsters while using the shortest time. After each round of combat, I will give you:\n\nTask list from last round: ...\nCritique: ...\nMonster: The monsters you need to defeat.\n\nThe critique (if any) will tell you the subgoal list from the previous round 
and whether you should trim or add to it.\nRemember to refer to the critique to adjust your task list.\n\nYou must follow the following criteria:\n1) Return a Python list of subgoals that can be completed in order to complete the specified task.\n2) Each subgoal should start with "craft", do not return any other type of skills.\n3) Each subgoal should follow a concise format "craft [material type] [equipment type]" such as "craft stone sword" and "craft iron helmet".\n\nYou should only respond in JSON format as described below:\n["subgoal1", "subgoal2", "subgoal3", ...]\nEnsure the response can be parsed by Python `json.loads`, e.g.: no trailing commas, no single quotes, etc.\nThe [] format will be used for RE extraction, do not use this format for anything other than your plan list and no extrabrackets!
'''

input_text = f'[INST]<<SYS>>\n{system_prompt}\n<</SYS>>\n\n {user_prompt}\n[/INST]' 
result = pipe(input_text)
print(result[0]['generated_text'])

  [
"craft wooden sword",
"craft wooden helmet",
"craft wooden chestplate",
"craft wooden leggings",
"craft wooden boots",
"kill zombie"
]
